In [4]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets 

In [8]:
import pandas as pd
df=pd.DataFrame(X)

In [11]:
len(df['workclass'].fillna('Desconocido').unique())

10

In [7]:
categorical_columns=X.select_dtypes(include='object').columns
categorical_columns

Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object')

In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Input, Flatten
from tensorflow.keras.models import Model

inputs_var=[]
num_cat=[]
for col_cat in categorical_columns:
    input_var_temp=Input(shape=(1,))
    inputs_var.append(input_var_temp)
    cant_cat=len(df[col_cat].fillna('Desconocido').unique())
    num_cat.append(cant_cat)
    
num_cat

[10, 16, 7, 16, 6, 5, 2, 43]

In [ ]:




# Definir las entradas para cada variable categórica
input_var1 = Input(shape=(1,))
input_var2 = Input(shape=(1,))
input_var3 = Input(shape=(1,))

# Crear una capa de incrustación para cada variable categórica
embedding_var1 = Embedding(input_dim=num_categories_var1, output_dim=4)(input_var1)
embedding_var2 = Embedding(input_dim=num_categories_var2, output_dim=4)(input_var2)
embedding_var3 = Embedding(input_dim=num_categories_var3, output_dim=4)(input_var3)

# Aplanar las salidas de las incrustaciones para poder concatenarlas si es necesario
embedding_var1_flat = Flatten()(embedding_var1)
embedding_var2_flat = Flatten()(embedding_var2)
embedding_var3_flat = Flatten()(embedding_var3)

# Crear un modelo de ejemplo que solo sirve para ilustrar la creación de embeddings
model = Model(inputs=[input_var1, input_var2, input_var3], outputs=[embedding_var1_flat, embedding_var2_flat, embedding_var3_flat])

# Resumen del modelo
model.summary()


In [15]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential

# Número de categorías únicas
num_unique_categories = 10
# Dimensión deseada para los vectores de incrustación
embedding_dim = 2

input_var=Input(shape=(1,))
embedding_var = Embedding(input_dim=num_unique_categories, output_dim=embedding_dim)(input_var)

embedding_var_flat = Flatten()(embedding_var)

# Crear un modelo de ejemplo que solo sirve para ilustrar la creación de embeddings
model = Model(inputs=[input_var], 
              outputs=[embedding_var_flat])

# Resumen del modelo
model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 1)]               0         
                                                                 
 embedding_1 (Embedding)     (None, 1, 2)              20        
                                                                 
 flatten_1 (Flatten)         (None, 2)                 0         
                                                                 
Total params: 20 (80.00 Byte)
Trainable params: 20 (80.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()
le.fit(df['workclass'])
code_label=le.transform(df['workclass'])

model.predict(code_label)

1527/1527 [==============================] - 1s 603us/step


array([[ 0.02941727, -0.01578633],
       [-0.02410465,  0.00894975],
       [ 0.0196127 , -0.0441796 ],
       ...,
       [ 0.0196127 , -0.0441796 ],
       [ 0.0196127 , -0.0441796 ],
       [ 0.03259477, -0.02859756]], dtype=float32)

In [22]:
np.unique(model.predict(code_label),axis=0)

1527/1527 [==============================] - 1s 605us/step


array([[-0.04766003, -0.02391441],
       [-0.04603465,  0.00829161],
       [-0.04038285,  0.01254353],
       [-0.02410465,  0.00894975],
       [-0.01577254, -0.01334681],
       [-0.01189471,  0.03719584],
       [ 0.00079622, -0.02576407],
       [ 0.0196127 , -0.0441796 ],
       [ 0.02941727, -0.01578633],
       [ 0.03259477, -0.02859756]], dtype=float32)